In [1]:
!pip install openai
!pip install sklearn
!pip install gradio

In [2]:
import json
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
from sklearn.metrics.pairwise import cosine_similarity
import gradio as gr



GPT_MODEL = "gpt-3.5-turbo-0613"
EMBEDDING_MODEL = "text-embedding-ada-002"

openai.api_key = open("key.txt", "r").read().strip("\n")


In [3]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [4]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }

    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))

In [5]:
functions = [
    {
        "name": "get_bests_houses",
        "description": """Utiliza esta función para ayudar al cliente a encontrar las casas que mejor
        se adapten a sus necesidades y preferencias. Asegúrate de que cualquier valor monetario esté expresado en pesos chilenos. Si el
        usuario proporciona un valor en otra moneda, solicita que lo convierta antes de usarlo como parámetro.""",
        "parameters": {
            "type": "object",
            "properties": {
                "description": {
                    "type": "string",
                    "description": """Una descripción detallada de lo que el usuario busca en una casa.
                    Incluye información sobre preferencias personales, ubicación deseada y cualquier característica específica que desee.
                    No incluir valores monetarios aquí.""",
                },
                "min_price": {
                    "type": "number",
                    "description": """El precio mínimo o cantidad mínima de dinero que el usuario está dispuesto a pagar por una
                     casa. Debe estar en pesos chilenos.""",
                },
                "max_price" : {
                    "type": "number",
                    "description": """El precio máximo o cantidad máxima de dinero que el usuario está dispuesto a pagar por una casa. Debe estar
                    en pesos chilenos."""
                },
                "nRooms": {
                    "type": "number",
                    "description": """El número de habitaciones que el usuario espera que tenga la casa."""
                }
            },

            "required": ["description"],
        },
    },
    {
        "name": "simulacion_hipotecaria",
        "description" : """Utiliza esta función para simular un crédito hipotecario del cliente. Antes de ejecutar la función el cliente debe validar que los parámetros
        que ingresó y tu, como asistente virtual entendiste, son correctos. Si y solo si el cliente valida, se puede ejecutar esta función. No debes asumir nunca un valor.
        Si el valor de la casa está en UF, el cliente debe validarlo. Si el valor está en pesos, el cliente debe validarlo.""",
        "parameters": {
            "type": "object",
            "properties": {
                "unidad_moneda": {
                    "type": "number",
                    "description": """El tipo de moneda que está utilizando para realizar la simulación. Por ejemplo: UF, pesos chilenos, pesos colombianos."""
                },
                "valor_propiedad": {
                    "type": "number",
                    "description": """Valor de la propiedad que quiere simular. Por ejemplo: 1000 (si está en UF), 97800000 (si está en pesos), etc."""
                },
                "monto_pie": {
                    "type": "number",
                    "description": """El porcentaje en decimal de lo que se quiere pagar de pie para la propiedad. Este valor siempre debe estár en su versión decimal,
                    Si el usuario lo ofrece en su versión porcentual, debes transformarlo"""
                },
                "plazo_pago": {
                    "type": "number",
                    "description": """El plazo de pagos que el cliente quiere para realizar la simulación. Este valor está en años. Solo se pueden valores enteros."""
                }
            },
            "required": ["unidad_moneda", "valor_propiedad","monto_pie","plazo_pago"]
        }
    }
]


In [6]:
with open('casas.json', 'r') as file:
    casas = json.load(file)

# Vectorización de texto
Función que convierte texto en vectores.
En base a la posición en la que se encuentren los vectores dentro del plano, se puede ver el grado de similitud que tiene con otros vectores.

In [7]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def embedding_request(text):
    response = openai.Embedding.create(input=text, model=EMBEDDING_MODEL)
    return response

# Filtrado de casas


In [8]:
def rooms_filter(houses, nRooms):
  if nRooms is not None:
        houses = [house for house in houses if house['habitaciones'] == nRooms ]
  return houses

def price_filter(houses, min_price=0, max_price=None):
    if min_price is None:
        min_price = 0;
    houses = [house for house in houses if int(house['precio'].replace("$", "").replace(",", "")) >= min_price and (max_price is None or int(house['precio'].replace("$", "").replace(",", "")) <= max_price)]
    return houses

def description_filter (houses, description):
  emb_res_ref = embedding_request(description)
  emb_ref = emb_res_ref["data"][0]["embedding"]

  descriptions_json = [item['descripcion'] for item in houses]
  embeddings_results = []

  for house_description in descriptions_json:
      res_house = embedding_request(house_description)
      embedding_house = res_house["data"][0]["embedding"]
      embeddings_results.append(embedding_house)

  similarity_scores = cosine_similarity([emb_ref], embeddings_results)[0]

  sorted_houses = [(houses[i], similarity_scores[i]) for i in range(len(similarity_scores))]
  sorted_houses.sort(key=lambda x: x[1], reverse=True)
  return sorted_houses

In [9]:
def get_best_descriptions(props):
    print(props)
    min_price = props.get('min_price', None)
    max_price = props.get('max_price', None)
    filtered_houses = casas

    if max_price is not None:
        filtered_houses = price_filter(filtered_houses, min_price, max_price)

    nRooms = props.get('nRooms', None)

    if(nRooms is not None):
        filtered_houses = rooms_filter(filtered_houses, nRooms)

    description = props.get('description', None)

    if(len(filtered_houses) != 0):
        if(description is not None):
          filtered_houses = description_filter(filtered_houses, description)
        return filtered_houses
    else:
        return "No se encontraron casas con los parámetros indicados"




In [10]:
def simulacion_hipoteca(props):
    unidadMoneda = props.get('unidad_moneda', None)
    valorPropiedad = props.get('valor_propiedad', None)
    montoPie = props.get('monto_pie', None)
    plazoPago = props.get('plazo_pago', None)

    return "Simulación realizada. Por favor espere."


In [11]:
def chat_completion_with_function_execution(messages, functions=[None]):
    """This function makes a ChatCompletion API call with the option of adding functions"""
    response = chat_completion_request(messages, functions)
    print("Respuesta antes del error:")
    print(response)
    full_message = response.json()["choices"][0]
    if full_message["finish_reason"] == "function_call":
        print(f"Function generation requested, calling function")
        return call_functions(messages, full_message)

    else:
        print(f"Function not required, responding to user")
        return response.json()

def call_functions(messages, full_message):
    """Function calling function which executes function calls when the model believes it is necessary.
    Currently extended by adding clauses to this if statement."""
    if full_message["message"]["function_call"]["name"] == "get_bests_houses":
        parsed_output = json.loads(
            full_message["message"]["function_call"]["arguments"]
        )
        results = get_best_descriptions(parsed_output)
        messages.append({
            "role": "function",
            "name": full_message["message"]["function_call"]["name"],
            # "content": str(results),
            "content": str(results),
        })
        response = chat_completion_request(messages)
        return response.json()
    if full_message["message"]["function_call"]["name"] == "simulacion_hipoteca":
        parsed_output = json.loads(
            full_message["message"]["function_call"]["arguments"]
        )
        results = simulacion_hipoteca(parsed_output)
        messages.append({
            "role": "function",
            "name": full_message["message"]["function_call"]["name"],
            "content": str(results),
        })
        response = chat_completion_request(messages)
        return response.json()
    else:
        raise Exception("Function does not exist and cannot be called")

In [12]:
class Conversation:
    def __init__(self):
        self.conversation_history = []

    def add_message(self, role, content):
        message = {"role": role, "content": content}
        self.conversation_history.append(message)

    def display_conversation(self, detailed=False):
        role_to_color = {
            "system": "red",
            "user": "green",
            "assistant": "blue",
            "function": "magenta",
        }
        for message in self.conversation_history:
            print(
                colored(
                    f"{message['role']}: {message['content']}\n\n",
                    role_to_color[message["role"]],
                )
            )

In [13]:
paper_system_message = """Eres HSI o Home Search Inteligence. Eres un chatbot de asistencia
que se encarga de encontrar la mejor casa para un usuario y también simular créditos hipotecarios.
Al ejecutar una función no debes asumir parámetros nunca. Si el cliente no proporciona un valor que es requerido, debes señalarlo.
También, considera que si el usuario ingresa un precio, valor o cualquier tipo de unidad monetaria, debe señalar qué tipo de unidad está utilizando, como UF's o pesos chilenos"""

paper_conversation = Conversation()
paper_conversation.add_message("system", paper_system_message)
# paper_conversation.add_message("system", "Soy HSI o Home Search Intelligence, un chatbot de asistencia inmobiliaria especializado en simulación de créditos hipotecarios. Mi función es ayudarte a encontrar la mejor casa y proporcionarte información sobre créditos. No asumo datos y solo trabajo con valores en UF's o pesos chilenos. ¿En qué puedo ayudarte hoy?")
# paper_conversation.add_message("system", "Recuerda que como HSI, no tomo decisiones por ti. Siempre señalaré si necesito más información o si alguna entrada es requerida para realizar una simulación de crédito hipotecario. No dudes en proporcionar los valores en UF's o pesos chilenos para una mejor precisión en las respuestas.")
# paper_conversation.add_message("system", "Como asistente especializado en créditos hipotecarios, puedo responder preguntas sobre tasas de interés, plazos de pago y cuotas mensuales. Si tienes alguna duda o deseas simular un crédito, no dudes en consultarme. Recuerda que solo trabajo con valores en UF's o pesos chilenos. Estoy aquí para ayudarte en todo momento.")

In [14]:
def chatbot_response(user_input, history):
    history = history or []
    paper_conversation.add_message("user", user_input)
    chat_response = chat_completion_with_function_execution(
        paper_conversation.conversation_history, functions=functions
    )
    assistant_message = chat_response["choices"][0]["message"]["content"]
    paper_conversation.add_message("assistant", assistant_message)
    history.append((user_input, assistant_message))
    return history,history

In [ ]:
chatbot = gr.Chatbot()
iface = gr.Interface(
    chatbot_response,    # La función que maneja las respuestas del chatbot
    ["text", "state"],
    [chatbot, "state"],
    allow_flagging="never",
    title="Home Search Inteligence"
)
iface.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://525f9c890f59bde5d9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Respuesta antes del error:
<Response [200]>
Function not required, responding to user
Respuesta antes del error:
<Response [200]>
Function generation requested, calling function
{'description': 'Necesito una casa con el mejor precio posible'}
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://525f9c890f59bde5d9.gradio.live
